# Analyze metadata from Zenodo spreadsheets corpus
Code to calculate file sizes, analyze file names, and generate lists of record ids for file downloading and corpus sampling related to testing and interview sampling frames.

## Imports and functions

In [3]:
import re
from datetime import datetime
import json
import pandas as pd
import math
import numpy as np
import edtf
from decimal import Decimal
import matplotlib.pyplot as plt
from edtf import parse_edtf, struct_time_to_date
from caseutil import get_cases
import regex
from langdetect import detect_langs
from langdetect.lang_detect_exception import LangDetectException
from langdetect import DetectorFactory # Important for consistency
import swifter
from itertools import combinations
import csv

In [ ]:
#function by Peter F: https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python
#Zenodo seems to calculate based on SI=True
def format_bytes(bytes, unit, SI=False):
    """
    Converts bytes to common units such as kb, kib, KB, mb, mib, MB

    Parameters
    ---------
    bytes: int
        Number of bytes to be converted

    unit: str
        Desired unit of measure for output


    SI: bool
        True -> Use SI standard e.g. KB = 1000 bytes
        False -> Use JEDEC standard e.g. KB = 1024 bytes

    Returns
    -------
    str:
        E.g. "7 MiB" where MiB is the original unit abbreviation supplied
    """
    if unit.lower() in "b bit bits".split():
        return f"{bytes*8} {unit}"
    unitN = unit[0].upper()+unit[1:].replace("s","")  # Normalised
    reference = {"Kb Kib Kibibit Kilobit": (7, 1),
                 "KB KiB Kibibyte Kilobyte": (10, 1),
                 "Mb Mib Mebibit Megabit": (17, 2),
                 "MB MiB Mebibyte Megabyte": (20, 2),
                 "Gb Gib Gibibit Gigabit": (27, 3),
                 "GB GiB Gibibyte Gigabyte": (30, 3),
                 "Tb Tib Tebibit Terabit": (37, 4),
                 "TB TiB Tebibyte Terabyte": (40, 4),
                 "Pb Pib Pebibit Petabit": (47, 5),
                 "PB PiB Pebibyte Petabyte": (50, 5),
                 "Eb Eib Exbibit Exabit": (57, 6),
                 "EB EiB Exbibyte Exabyte": (60, 6),
                 "Zb Zib Zebibit Zettabit": (67, 7),
                 "ZB ZiB Zebibyte Zettabyte": (70, 7),
                 "Yb Yib Yobibit Yottabit": (77, 8),
                 "YB YiB Yobibyte Yottabyte": (80, 8),
                 }
    key_list = '\n'.join(["     b Bit"] + [x for x in reference.keys()]) +"\n"
    if unitN not in key_list:
        raise IndexError(f"\n\nConversion unit must be one of:\n\n{key_list}")
    units, divisors = [(k,v) for k,v in reference.items() if unitN in k][0]
    if SI:
        divisor = 1000**divisors[1]/8 if "bit" in units else 1000**divisors[1]
    else:
        divisor = float(1 << divisors[0])
    value = bytes / divisor
    return value

In [ ]:
#function using the edtf library to parse the metadata.publication_date field, since it is in EDTF form.
#finds the earlier date if there is a date range and extracts the year
def parseable_date(x):
    y = parse_edtf(x)
    z = struct_time_to_date(y.lower_strict()).year
    return z

In [ ]:
#function that uses caseutil to try to guess the case of the string
def filename_cases(x):
    cases = get_cases(x)
    cases_string = ';'.join(cases)
    return cases_string

In [ ]:
#https://www.w3schools.in/python/examples/language-detection
#https://stackoverflow.com/questions/39142778/how-to-determine-the-language-of-a-piece-of-text
#original takes forever to run
#asked Google Gemini to optimize; Gemini also recommended using swifter
#prompt: Can you optimize this Python function that uses the langdetect library? 
# and then I am implementing this in a large pandas dataframe. What changes would you make so that it processes faster?

# from langdetect import detect_langs
# from langdetect.lang_detect_exception import LangDetectException
# from langdetect import DetectorFactory # Important for consistency

# Set a seed once at the beginning of your script/application
DetectorFactory.seed = 0

def detect_lang(text):
    if not isinstance(text, str) or not text.strip():
        return 'no-lang' # Handle non-string or empty/whitespace-only input early

    try:
        languages = detect_langs(text)
        # Use a list comprehension for efficient string building
        lang_parts = [f"{lang.lang};{lang.prob:.4f}" for lang in languages] # Format probability
        return '|'.join(lang_parts)
    except LangDetectException:
        return 'no-lang'
    except Exception as e:
        # Log unexpected errors for debugging, but return 'no-lang' for consistency
        # import logging
        # logging.error(f"Unexpected error for text '{text}': {e}")
        return 'no-lang'

# original attempt:
#def detect_lang(x):
#     try:
#         languages = detect_langs(x)
#         lang_string = ''
#         for language in languages:
#             lang_code = language.lang
#             #print(lang_code)
#             lang_prob = str(language.prob)
#             #print(lang_prob)
#             lang_string += lang_code +';'+lang_prob +'|'
#         return lang_string
#     except Exception as e:
#         return 'no-lang'
# Sample Output:
# en 0.50 (English)
# hi 0.50 (Hindi)

In [ ]:
#https://stackoverflow.com/questions/25628973/capturing-named-groups-in-regex-with-re-findall
#used Google Gemini Flash 2.5 to modify this code and use sets instead of lists and to concatenate the string
def filename_capitalization(x):

    match = regex.findall(r'(?P<lower>\p{Ll})|(?P<upper>\p{Lu})|(?P<digit>\d)', x, re.UNICODE)
    capitalization_types=set()
    
    for tup in match:
        if tup[0] != '':
            capitalization_types.add('lower')
        if tup[1] != '':
            capitalization_types.add('upper')
        if tup[2] != '':
            capitalization_types.add('digit')
    capitalization_list=list(capitalization_types)
    capitalization_list.sort()
    concatenated_string = ';'.join(capitalization_list)
    return concatenated_string

In [ ]:
#set the global option to prevent scientific notation in aggregation and other derivative dataframes.
pd.options.display.float_format = '{:20,.2f}'.format

## Load data

In [ ]:
#load the API results, which filtered filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb OR ots OR xlsm OR xltm OR xltx OR ogw) AND doi:zenodo AND created:{date}
with open('zenodo_API_metadata_results_2025-11-05_03-38_PM.json', 'r') as f:
#with open('zenodo_snippet.json', 'r') as f:
    data = f.read()
zenodo_dict = json.loads(data)

## Create dataframes for metadata

In [ ]:
#create a dataframe that contains all of the JSON API result, including some fields that contain nested JSON
zenodo_df = pd.json_normalize(zenodo_dict)

In [ ]:
zenodo_df.info()

### Find duplicates

In [ ]:
# how many unique ids are there?
len(zenodo_df['id'].unique())

In [ ]:
#does not work because some columns contain lists.
#zenodo_df.duplicated(keep=False).sort_values('id')

In [ ]:
#find duplicate records, comparing all fields. since come columns contain lists, have to treat df as string
##https://stackoverflow.com/questions/43855462/pandas-drop-duplicates-method-not-working-on-dataframe-containing-lists
zenodo_dupes_1 = zenodo_df.loc[zenodo_df.astype(str).duplicated(keep=False)].sort_values('id')

In [ ]:
#find duplicate records, comparing only id
zenodo_dupes_2 = zenodo_df.loc[zenodo_df.duplicated(subset='id', keep=False)].sort_values('id')

In [ ]:
#export files so the two lists of duplicates can be compared, and can figure out why some duplicates are not being found when all rows are compared

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_dupes_1.to_csv(f'zenodo_dupes_1_{filetime}.csv')

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_dupes_2.to_csv(f'zenodo_dupes_2_{filetime}.csv')

In [ ]:
#turns out it is because of download stats!
#get a list of column headers that excludes the stats cols

In [ ]:
zenodo_df_columns = zenodo_df.columns.values.tolist()

In [ ]:
zenodo_df_columns

In [ ]:
#https://pythonguides.com/remove-multiple-elements-from-a-list-in-python/
#remove the stats columns from the list of columns, so this list can be passed to drop_duplicates
cols_to_remove = ['stats.downloads','stats.unique_downloads','stats.views','stats.unique_views','stats.version_downloads','stats.version_unique_downloads','stats.version_unique_views','stats.version_views']

zenodo_df_columns_filtered = [item for item in zenodo_df_columns if item not in cols_to_remove]
print(zenodo_df_columns_filtered)


In [ ]:
# get the duplicate rows, using all columns except the stats ones to determine what's a dupe
zenodo_df.loc[zenodo_df.astype(str).duplicated(subset=zenodo_df_columns_filtered,keep=False)].sort_values('id')

In [ ]:
zenodo_df_deduped = zenodo_df.loc[zenodo_df.astype(str).drop_duplicates(subset=zenodo_df_columns_filtered).index].reset_index(drop=True)

In [ ]:
zenodo_df_deduped

### Export lists for zenodo_get downloader

In [ ]:
#entire list of record ids

In [ ]:
#sample of 20 record ids

### Add columns used for analysis

In [ ]:
#first, export the publication dates to a CSV so they can be examined in OpenRefine and issues like whitespace or non-compliant ETDF dates found.
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_pub_dates = zenodo_df_deduped[['id','metadata.publication_date']]
zenodo_pub_dates.to_csv(f'zenodo_pub_dates_{filetime}.csv')

In [ ]:
#then, fix and replace any dates that have issues.
#two records were found to have issues with dates: 14215531 and 14030741
#round 2 zenodo_df['metadata.publication_date'] = zenodo_df['metadata.publication_date'].str.strip().replace('02_08_2024','2024-08-02')
zenodo_df_deduped['metadata.publication_date'] = zenodo_df_deduped['metadata.publication_date'].str.strip()

In [ ]:
#check the records that had date issues before to make sure they are fixed: 13169677,14030741, 14215531
zenodo_df_deduped.loc[zenodo_df_deduped['id'] == 14030741]['metadata.publication_date']

In [ ]:
#create a column for the earliest publication year by using the EDTF library
zenodo_df_deduped['earliest_pub_year'] = zenodo_df_deduped['metadata.publication_date'].apply(lambda x: parseable_date(x))

In [ ]:
#convert the date created column to a datetime format
zenodo_df_deduped['created'] = pd.to_datetime(zenodo_df_deduped['created'], format='ISO8601')

In [ ]:
#create a column for the year the record was created.
zenodo_df_deduped['year_created']=pd.DatetimeIndex(zenodo_df_deduped['created']).year

## Create dataframes for files

In [ ]:
#use json_normalize to create dataframes. each record represents a file from the metadata record.
#https://stackoverflow.com/questions/52085169/valueerror-conflicting-metadata-name-name-need-distinguishing-prefix-in-pandas
#since there are two JSON fields with key 'id', need to distinguish them using record_prefix
zenodo_files_df = pd.json_normalize(zenodo_dict, record_path=[['files']],
                                       meta=['id'], record_prefix='file'
                                      )

In [ ]:
zenodo_files_df

In [ ]:
#double check for duplicates
zenodo_files_df[zenodo_files_df.duplicated(keep=False)].sort_values('filekey')

In [ ]:
zenodo_files_df[zenodo_files_df.duplicated(subset='fileid', keep=False)].sort_values('filekey')

In [ ]:
zenodo_files_df_deduped = zenodo_files_df.drop_duplicates().reset_index(drop=True)

In [ ]:
zenodo_files_df_deduped.info()

## Drop the records did not download
drop the records that contain files that couldn't be downloaded, because of a change in record status, etc

In [ ]:
#for now, create copies for testing purposes
zenodo_df_testcopy = zenodo_df.copy()
zenodo_files_df_testcopy = zenodo_files_df.copy()

In [ ]:
# import inventory dataframe
#https://www.geeksforgeeks.org/python-pandas-series-from_csv/
file_inventory_df = pd.read_csv('file_inventory_2025-06-26_02-38_PM.csv')
file_inventory_df


In [ ]:
file_inventory_df = file_inventory_df.dropna().reset_index(drop=True)

In [ ]:
#here want to get just a list of the recordids aka subdirectory_1, so need to g
file_inventory_recordids = file_inventory_df['Subdirectory_Level_1'].unique()
file_inventory_recordids



In [ ]:
#https://stackoverflow.com/questions/50449088/check-if-value-from-one-dataframe-exists-in-another-dataframe
#https://www.askpython.com/python-modules/pandas/pandas-isin
#https://codemax.app/snippet/checking-if-elements-in-an-array-exist-in-a-pandas-dataframe-in-python/

#Df1.assign(InDf2=Df1.name.isin(Df2.IDs).astype(int))

zenodo_df_testcopy = zenodo_df_testcopy.assign(in_file_inventory=zenodo_df_testcopy['id'].isin(file_inventory_recordids))
# fileinventory_record_filter = zenodo_df_testcopy['id'].isin(file_inventory_recordids)
# zenodo_df_testcopy[fileinventory_record_filter]

#zenodo_df_testcopy['in_file_inventory']


In [ ]:
zenodo_df_testcopy = zenodo_df_testcopy.loc[zenodo_df_testcopy['in_file_inventory']==True]


## Analysis at the record level

### Record years plots

In [ ]:
# The distribution of records across years
# need to plot this in a more interesting way and ensure x axis marks properly show years
# https://stackoverflow.com/questions/45108455/graphing-number-of-rows-over-time-in-pandas
record_created_years_graph = zenodo_df_deduped.groupby('year_created').size().plot(kind='bar', color='purple', ylabel='Number of records', xlabel='Year record created', grid=False)
#annotate bars
record_created_years_graph.bar_label(record_created_years_graph.containers[0])

In [ ]:
# The distribution of records across years
# need to plot this in a more interesting way and ensure x axis marks properly show years
record_published_years_graph = zenodo_df_deduped.groupby('earliest_pub_year').size().plot(kind='bar', color='maroon', ylabel='Number of records', xlabel='Publication year (earliest)', grid=False, figsize=(10,12))
record_published_years_graph.bar_label(record_published_years_graph.containers[0])

### Related identifiers analysis

In [ ]:
# create a dataframe of records related to some other research entity (deposit, article, etc) in metadata.related_identifiers
zenodo_related_identifiers = zenodo_df_deduped[['id','metadata.related_identifiers']].dropna().rename({"metadata.related_identifiers": "related_identifiers"}, axis="columns")
zenodo_related_identifiers

In [ ]:
zenodo_related_identifiers = zenodo_related_identifiers.reset_index(drop=True)

In [ ]:
zenodo_related_identifiers

In [ ]:
#take a look at the contents of a cell
zenodo_related_identifiers['related_identifiers'][4]

In [ ]:
#put each related identifier, which is a dictionary/JSON object, in its own row
#https://www.geeksforgeeks.org/convert-list-like-column-elements-to-separate-rows-in-pandas/
zenodo_related_identifiers_melt = zenodo_related_identifiers.related_identifiers.apply(pd.Series).merge(zenodo_related_identifiers, right_index = True, left_index = True).drop(['related_identifiers'], axis = 1).melt(id_vars = ['id'], value_name = 'related_identifiers').drop('variable', axis=1).dropna()

In [ ]:
zenodo_related_identifiers_melt = zenodo_related_identifiers_melt.reset_index(drop=True)

In [ ]:
zenodo_related_identifiers_melt

In [ ]:
zenodo_related_identifiers['id']

In [ ]:
#put each identifier element in its own column
#https://datascientyst.com/normalize-json-dict-new-columns-pandas/
#https://stackoverflow.com/questions/46391291/how-to-convert-json-data-inside-a-pandas-column-into-new-columns
zenodo_relations = pd.json_normalize(zenodo_related_identifiers_melt['related_identifiers'])
#.merge(zenodo_related_identifiers_melt['id'], right_index=True)

In [ ]:
zenodo_relations

In [ ]:
zenodo_relations.reset_index(drop=True)

In [ ]:
# merge the original related identifiers dataframe with the one that puts each element in its own column
zenodo_related_identifiers_all = pd.concat([zenodo_relations, zenodo_related_identifiers_melt['id']], axis=1)

In [ ]:
zenodo_related_identifiers_all

In [ ]:
zenodo_related_identifiers_sankey = zenodo_related_identifiers_all[['relation','scheme']]

In [ ]:
zenodo_related_identifiers_sankey

In [ ]:
#consider consolidating relationships to be bidirectional
zenodo_related_identifiers_sankey['relation_consolidated'] = zenodo_related_identifiers_sankey['relation'].replace({'isSupplementTo':'supplementary','isSupplementedBy':'supplementary','isDerivedFrom':'sourcing','isSourceOf':'sourcing','isContinuedBy':'continuation','continues':'continuation','isCitedBy':'citation','cites':'citation','references':'referencing','isReferencedBy':'referencing','isDescribedBy':'description','describes':'description','documents':'documentation','isDocumentedBy':'documentation','isCompiledBy':'compilation','compiles':'compilation','hasPart':'partitive','isPartOf':'partitive','isMetadataFor':'metadata','hasMetadata':'metadata','requires':'requirement','isRequiredBy':'requirement','isVersionOf':'versioning','hasVersion':'versioning','isPreviousVersionOf':'superceding','isNewVersionOf':'superceding','obsoletes':'obsolesence','isObsoletedBy':'obsolesence','isReviewedBy':'reviewing','reviews':'reviewing'})

In [ ]:
#https://www.statology.org/pandas-groupby-to-dataframe/
zenodo_related_identifiers_sankey_count = zenodo_related_identifiers_sankey.groupby(['relation_consolidated','scheme']).value_counts().sort_values(ascending=False).reset_index(name='count')

In [ ]:
zenodo_related_identifiers_sankey_count

In [ ]:
#used Gemini for sankey prep: https://gemini.google.com/app/596c5c70ccb22615

# --------------------------------------------------------------------------------------
# --- 3. SANKEY CHART PREPARATION: Generating Labels and Indices ---
# --------------------------------------------------------------------------------------

# 3A. Create the unique list of all nodes (LABELS) for the Sankey chart
all_nodes = pd.concat([zenodo_related_identifiers_sankey_count['relation_consolidated'], zenodo_related_identifiers_sankey_count['scheme']]).unique()
df_labels = pd.Series(all_nodes).rename('Sankey_Labels')

# Create a mapping dictionary: {ID_string: index_number}
label_map = {label: i for i, label in enumerate(all_nodes)}

# 3B. Map Source and Target IDs to their corresponding indices (Source/Target Index Lists)
df_source_indices = zenodo_related_identifiers_sankey_count['relation_consolidated'].map(label_map).rename('Source_Index')
df_target_indices = zenodo_related_identifiers_sankey_count['scheme'].map(label_map).rename('Target_Index')

# Output 4: List of Weights for each relationship (Plotly Values)
df_weights_only = zenodo_related_identifiers_sankey_count['count'].reset_index(drop=True).rename('Weight_List')


print("--- 3. SANKEY OUTPUT A: Unique List of All Labels (Nodes) ---")
print("This is the 'label' argument for your Plotly Sankey chart.")
print(df_labels)
print(f"\nTotal unique nodes: {len(df_labels)}\n")
print("-" * 50 + "\n")

print("--- 4. SANKEY OUTPUT B: Source Indices ---")
print("The 'source' argument for Plotly. Maps each source ID to its index in the Labels list.")
print(df_source_indices)
print(f"\nType: {type(df_source_indices)}\n")
print("-" * 50 + "\n")


print("--- 5. SANKEY OUTPUT C: Target Indices ---")
print("The 'target' argument for Plotly. Maps each target ID to its index in the Labels list.")
print(df_target_indices)
print(f"\nType: {type(df_target_indices)}\n")
print("-" * 50 + "\n")

print("--- 6. SANKEY OUTPUT D: Weights (Values) ---")
print("The 'value' argument for Plotly (frequency of the relationship).")
print(df_weights_only)
print(f"\nType: {type(df_weights_only)}\n")
print("-" * 50 + "\n")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement="freeform",
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.1),
      label = df_labels,
      color = "blue"
    ),
    link = dict(
      source = df_source_indices, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = df_target_indices,
      value = df_weights_only
  ))])

fig.update_layout(title_text="Relations and Schemes", font_size=10, autosize=False,
    width=800,
    height=800)
fig.show()

In [ ]:
#get a sense of the number of different kinds of relationships present
#https://www.statology.org/pandas-value_counts-sort/
#zenodo_related_identifiers_all.groupby(['relation']).count().sort_values(by='identifier',ascending=False)
zenodo_related_identifiers_all.groupby(['relation'])['relation'].value_counts().sort_values(ascending=False)

In [ ]:
len(zenodo_related_identifiers_all.groupby(['relation'])['relation'].value_counts().sort_values(ascending=False))

In [ ]:
#get a sense of the identifier schemes in use
#zenodo_related_identifiers_all.groupby(['scheme']).count().sort_values(by='identifier',ascending=False)
zenodo_related_identifiers_all.groupby(['scheme'])['scheme'].value_counts().sort_values(ascending=False)

In [ ]:
#zenodo_related_identifiers_all.groupby(['id']).count().sort_values(by='identifier',ascending=False)
#https://stackoverflow.com/questions/50824493/pandas-groupby-filter-on-count
#how many records have 10 or more related identifiers associated?
zenodo_related_identifiers_all.groupby(['id'])['id'].value_counts()[lambda x: x>=100].sort_values(ascending=False)

In [ ]:
zenodo_related_identifiers_all.groupby(['id']).count().plot(kind='hist')

In [ ]:
#put the related identifiers in their own file so they can be looked at in OpenRefine
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_related_identifiers_all.to_csv(f'zenodo_related_identifiers_{filetime}.csv')

### Keyword analysis

In [ ]:
# look into extracting and analyzing keywords to get at discipline
zenodo_keywords_df = zenodo_df_deduped[['id','metadata.keywords']].dropna().explode('metadata.keywords')

In [ ]:
zenodo_keywords_df

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_keywords_df.to_csv(f'zenodo_keywords_{filetime}.csv')

In [ ]:
zenodo_df_deduped['id','metadata.subjects']].dropna()
#.dropna().explode('metadata.keywords')

## Add columns to the dataframes that will be used for analysis

In [ ]:
#add a column for filename length
zenodo_files_df_deduped['filekey_len'] = zenodo_files_df_deduped['filekey'].str.len()

In [ ]:
# add a column for the file extensions
#https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html
zenodo_files_df_deduped['filetype'] = zenodo_files_df_deduped['filekey'].str.extract(r'(?:.*)(\..+)$')

In [ ]:
zenodo_files_df_deduped['filetype'] = zenodo_files_df_deduped['filetype'].str.lower()

In [ ]:
#look for files that don't have a proper file extension
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filetype'].isna()==True]

In [ ]:
#for those that don't have an extension, fill column with word 'none'
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filetype'].isna()==True, 'filetype'] = zenodo_files_df_deduped['filetype'].fillna('none')

In [ ]:
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filetype'] == 'none']

In [ ]:
#add a column for the filename but without extension
zenodo_files_df_deduped['filename_noext'] = zenodo_files_df_deduped['filekey'].str.extract(r'(.*)(?:\..+)$')

In [ ]:
zenodo_files_df_deduped['filename_noext']

In [ ]:
#look for the filenames that couldn't be parsed because there was no extension
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].isna()==True]

In [ ]:
#copy the filekey over to the filename with no extension column
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].isna()==True, 'filename_noext'] = zenodo_files_df_deduped['filekey']

In [ ]:
#double check that the copying worked
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].isna()==True]

In [ ]:
#add a column for filename_noext length
zenodo_files_df_deduped['filename_noext_len'] = zenodo_files_df_deduped['filename_noext'].str.len()

In [ ]:
#add columns to dataframe that convert the file size in bytes to MB and GB respectively
zenodo_files_df_deduped['filesize_mb'] = zenodo_files_df_deduped['filesize'].apply(lambda x: format_bytes(x, 'MB', SI=True))

In [ ]:
zenodo_files_df_deduped['filesize_gb'] = zenodo_files_df_deduped['filesize'].apply(lambda x: format_bytes(x, 'GB', SI=True))

In [ ]:
zenodo_files_df_deduped

## Create a dataframe of just the spreadsheet files

In [ ]:
#make a list of spreadsheet file extensions
spreadsheet_filetypes = ['.csv','.tsv','.xls','.xlsb','.xlsx','.xltx','.ods','.ots','.xlsm','.xltm','.xltx','.ogw']

In [ ]:

#https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
zenodo_spreadsheet_files_df = zenodo_files_df_deduped.query('filetype in @spreadsheet_filetypes')

In [ ]:
zenodo_spreadsheet_files_df.info()

In [ ]:
zenodo_spreadsheet_files_df = zenodo_spreadsheet_files_df.reset_index(drop=True)

### Check the file inventory against the list of spreadsheet files to make sure all downloaded OK

In [ ]:
#want to compare both the id and the filename columns in both dataframes to find out which ones from file_inventory_df are NOT in zenodo_files_df
#https://stackoverflow.com/questions/48647534/find-difference-between-two-data-frames
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.compare.html
#need to move this after creating the dataframe of just the spreadsheet files.
zenodo_spreadsheet_files_df_testcopy = zenodo_spreadsheet_files_df.copy()

In [ ]:
zenodo_spreadsheet_files_df_testcopy_compare = zenodo_spreadsheet_files_df_testcopy[['id','filekey']]

In [ ]:
file_inventory_df_compare = file_inventory_df[['Subdirectory_Level_1','File']].set_axis(['id', 'filekey'], axis=1)

In [ ]:
file_inventory_df_compare

In [ ]:
# https://stackoverflow.com/questions/48647534/find-difference-between-two-data-frames
#df1.merge(df2,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
zenodo_spreadsheet_files_downloaded = zenodo_spreadsheet_files_df_testcopy_compare.merge(file_inventory_df_compare,indicator=True,how='left').loc[lambda x : x['_merge']=='both']

In [ ]:
zenodo_spreadsheet_files_downloaded

In [ ]:
#https://stackoverflow.com/questions/48864923/select-certain-rows-by-index-of-another-dataframe
#df[df.index.isin(df1.index)]
zenodo_spreadsheet_files_df_testcopy_check = zenodo_spreadsheet_files_df_testcopy[zenodo_spreadsheet_files_df.index.isin(zenodo_spreadsheet_files_downloaded.index)]

In [ ]:
len(zenodo_spreadsheet_files_df_testcopy_check.index)

In [ ]:
len(zenodo_spreadsheet_files_downloaded.index)

## Analysis of size of the corpus: counts and file size

In [ ]:
#check for files that are over 50 GB
zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filesize_gb'] > 50]

In [ ]:
#get some stats about the different file types. reveals that there are some poorly formed file names without proper extensions.
zenodo_files_df_deduped.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
#take a look at the spreadsheet files dataframe
#275424
zenodo_spreadsheet_files_df.info()

### Get the total filesize of all spreadsheets


In [ ]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
spreadsheet_filesize_total = zenodo_spreadsheet_files_df['filesize'].sum()
print(spreadsheet_filesize_total)

In [ ]:
#format the total filesize of all spreadsheets in TB
format_bytes(spreadsheet_filesize_total, 'TB', SI=True)

In [ ]:
#get the total number of spreadsheet files
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
spreadsheet_files_count = len(zenodo_spreadsheet_files_df.index)
print(spreadsheet_files_count)

In [ ]:
zenodo_spreadsheet_files_df.describe()

### Get some basic stats about the spreadsheet files broken out by filetype


In [ ]:
#get some basic stats about the spreadsheet files.
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
#get some basic stats about the spreadsheet files, in GB.
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_gb': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
#get some basic stats about the spreadsheet files, in MB.
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
#get just the total size of all files by filetype
#https://dnmtechs.com/calculating-percentage-of-total-with-pandas-groupby-in-python-3/
zenodo_spreadsheet_files_size_grouped = zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize': ['sum']})
zenodo_spreadsheet_files_size_grouped

In [ ]:
#get just the total size in GB of all files by filetype
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_gb': ['sum']})

In [ ]:
#get just the total count of all files by filetype
zenodo_spreadsheet_files_count_grouped = zenodo_spreadsheet_files_df.groupby('filetype').agg({'id':'count'}).sort_values(by='id', ascending=False )

In [ ]:
zenodo_spreadsheet_files_count_grouped

In [ ]:
zenodo_spreadsheet_files_df.groupby('filetype')['filetype'].value_counts().sort_values().plot(kind='pie')

In [ ]:
zenodo_spreadsheet_files_count_grouped.plot(kind='pie', subplots=True, figsize=(8,8))

In [ ]:
#get the sizes of each filetype as percentages
zenodo_spreadsheet_files_size_grouped.transform(lambda x: (x / spreadsheet_filesize_total) * 100)
#.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

In [ ]:
#get the count of each filetype as percentages
zenodo_spreadsheet_files_count_grouped.transform(lambda x: (x / spreadsheet_files_count) * 100)

In [ ]:
#isolate just the Excel files of various types
zenodo_spreadsheet_files_count_grouped.loc[['.xlsx','.xls','.xlsm','.xlsb','.xltx','.xltm']]

In [ ]:
#get the total number of all the Excel files of various types
zenodo_spreadsheet_files_count_grouped.loc[['.xlsx','.xls','.xlsm','.xlsb','.xltx','.xltm']].sum()

In [ ]:
#get the total percentage of all the Excel files of various types
zenodo_spreadsheet_files_count_grouped.loc[['.xlsx','.xls','.xlsm','.xlsb','.xltx','.xltm']].sum().transform(lambda x: (x / spreadsheet_files_count) * 100)

In [ ]:
#get the total number of all tabular files
zenodo_spreadsheet_files_count_grouped.loc[['.csv','.tsv']].sum()

In [ ]:
#get the total percentage of all tabular files
zenodo_spreadsheet_files_count_grouped.loc[['.csv','.tsv']].sum().transform(lambda x: (x / spreadsheet_files_count) * 100)

### Various plots of file size and counts

In [ ]:
plt.hist(zenodo_spreadsheet_files_df[zenodo_spreadsheet_files_df['filetype']=='.ods'].filesize_gb, bins=100)

In [ ]:
plt.hist(zenodo_spreadsheet_files_df.filesize_gb, bins=30)

In [ ]:
zenodo_spreadsheet_files_groups = zenodo_spreadsheet_files_df.groupby('filetype')

In [ ]:
zenodo_spreadsheet_files_df.groupby('filetype')['filesize_gb'].plot(legend=True)

In [ ]:
zenodo_spreadsheet_files_groups.get_group('.csv')['filesize_gb']

In [ ]:
fig, ax = plt.subplots(11,1,figsize=(25,30), squeeze=True)

ax[0].hist(zenodo_spreadsheet_files_groups.get_group('.csv')['filesize_gb'], bins=50)
ax[0].set_title('CSV size in GB')
ax[1].hist(zenodo_spreadsheet_files_groups.get_group('.tsv')['filesize_gb'], bins=50)
ax[1].set_title('TSV size in GB')
ax[2].hist(zenodo_spreadsheet_files_groups.get_group('.xlsx')['filesize_gb'], bins=50)
ax[2].set_title('XLSX size in GB')
ax[3].hist(zenodo_spreadsheet_files_groups.get_group('.xls')['filesize_gb'], bins=50)
ax[3].set_title('XLS size in GB')
ax[4].hist(zenodo_spreadsheet_files_groups.get_group('.xlsb')['filesize_gb'], bins=50)
ax[4].set_title('XLSB size in GB')
ax[5].hist(zenodo_spreadsheet_files_groups.get_group('.xltx')['filesize_gb'], bins=50)
ax[5].set_title('XLTX size in GB')
ax[6].hist(zenodo_spreadsheet_files_groups.get_group('.xlsm')['filesize_gb'], bins=50)
ax[6].set_title('XLSM size in GB')
ax[7].hist(zenodo_spreadsheet_files_groups.get_group('.xltm')['filesize_gb'], bins=50)
ax[7].set_title('XLTM size in GB')
ax[8].hist(zenodo_spreadsheet_files_groups.get_group('.ods')['filesize_gb'], bins=50)
ax[8].set_title('ODS size in GB')
ax[9].hist(zenodo_spreadsheet_files_groups.get_group('.ots')['filesize_gb'], bins=50)
ax[9].set_title('OTS size in GB')
ax[10].hist(zenodo_spreadsheet_files_groups.get_group('.ogw')['filesize_gb'], bins=50)
ax[10].set_title('OGW size in GB')

In [ ]:
fig, ax = plt.subplots(11,1,figsize=(25,30), squeeze=True)

ax[0].hist(zenodo_spreadsheet_files_groups.get_group('.csv')['filesize_mb'], bins=50)
ax[0].set_title('CSV size in MB')
ax[1].hist(zenodo_spreadsheet_files_groups.get_group('.tsv')['filesize_mb'], bins=50)
ax[1].set_title('TSV size in MB')
ax[2].hist(zenodo_spreadsheet_files_groups.get_group('.xlsx')['filesize_mb'], bins=50)
ax[2].set_title('XLSX size in MB')
ax[3].hist(zenodo_spreadsheet_files_groups.get_group('.xls')['filesize_mb'], bins=50)
ax[3].set_title('XLS size in MB')
ax[4].hist(zenodo_spreadsheet_files_groups.get_group('.xlsb')['filesize_mb'], bins=50)
ax[4].set_title('XLSB size in MB')
ax[5].hist(zenodo_spreadsheet_files_groups.get_group('.xltx')['filesize_mb'], bins=50)
ax[5].set_title('XLTX size in MB')
ax[6].hist(zenodo_spreadsheet_files_groups.get_group('.xlsm')['filesize_mb'], bins=50)
ax[6].set_title('XLSM size in MB')
ax[7].hist(zenodo_spreadsheet_files_groups.get_group('.xltm')['filesize_mb'], bins=50)
ax[7].set_title('XLTM size in MB')
ax[8].hist(zenodo_spreadsheet_files_groups.get_group('.ods')['filesize_mb'], bins=50)
ax[8].set_title('ODS size in MB')
ax[9].hist(zenodo_spreadsheet_files_groups.get_group('.ots')['filesize_mb'], bins=50)
ax[9].set_title('OTS size in MB')
ax[10].hist(zenodo_spreadsheet_files_groups.get_group('.ogw')['filesize_mb'], bins=50)
ax[10].set_title('OGW size in MB')

## Analysis of file names

### Files were deposited in multiple formats
determined by looking for the same file name with different extensions

In [ ]:
zenodo_files_df_deduped

In [ ]:
#sort the dataframe by id, filename, and filetype
zenodo_files_df_sorted = zenodo_files_df_deduped.sort_values(by=['id','filename_noext','filetype'])

In [ ]:
zenodo_files_df_sorted

In [ ]:
#if a filename with no extension has multiple filetypes, concatenate those in the filetype column
#https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
zenodo_files_df_sorted.groupby(['id','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

In [ ]:
#if a filename with no extension has multiple filetypes, concatenate those in the filetype column
#now put that groupby in its own dataframe
#https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
filename_counts = zenodo_files_df_sorted.groupby(['id','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

In [ ]:
filename_counts

In [ ]:
filename_counts.columns

In [ ]:
#reset some indexes and column names
filename_counts.index.name = None
filename_counts.columns=['counts', 'filetype']

In [ ]:
filename_counts = filename_counts.rename_axis(['id','filename_noext']).reset_index()

In [ ]:
filename_counts

In [ ]:
#from the dataframe that counts filenames with no extensions, find the ones greater than 1
filename_duplicates = filename_counts.loc[filename_counts['counts']>1]

In [ ]:
filename_duplicates

In [ ]:
#export the dataframe of duplicate filenames so this can be looked at in OpenRefine
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

filename_duplicates.to_csv(f'filename_duplicates_{filetime}.csv', index=False)

In [ ]:
#now count how many instances of duplicate files occur per record
filename_duplicates_counts = filename_duplicates.groupby(['id']).agg({'counts':'count'}).sort_values(by='counts',ascending=False)

In [ ]:
filename_duplicates_counts

In [ ]:
filename_duplicates.loc[filename_duplicates['id']==14583147]

In [ ]:
filename_duplicates_counts.loc[filename_duplicates_counts['counts']>1]

In [ ]:
#find out how many records have multiple instances of duplicate filenames that are also spreadsheets
filename_duplicates_spreadsheets = filename_duplicates.loc[filename_duplicates['filetype'].str.contains(r'\.csv|\.tsv|\.xls[xbm]?|\.xlt[xm]?|\.ods|\.ots|\.ogw', regex=True, case=False)]

In [ ]:
filename_duplicates_spreadsheets

In [ ]:
filename_duplicates_spreadsheets_filetype_counts = filename_duplicates_spreadsheets.groupby('filetype',).count().sort_values(by='counts',ascending=False)

In [ ]:
filename_duplicates_spreadsheets_filetype_counts

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

filename_duplicates_spreadsheets.to_csv(f'filename_duplicates_spreadsheets_{filetime}.csv', index=False)

In [ ]:
#filename_duplicates_spreadsheets['filetype'] contains two or more items from spreadsheet_filetypes list
#used Google Gemini for help; it suggested str.count
filename_duplicates_spreadsheets_multispreadsheets = filename_duplicates_spreadsheets[filename_duplicates_spreadsheets['filetype'].str.count(r'\.csv|\.tsv|\.xls[xbm]?|\.xlt[xm]?|\.ods|\.ots|\.ogw')>= 2]

In [ ]:
filename_duplicates_spreadsheets_multispreadsheets

In [ ]:
#what are the combinations of spreadsheet filetypes? how many represent the good practice of open version of a proprietary format?
filename_duplicates_spreadsheets_multispreadsheets.groupby('filetype')['filetype'].value_counts().sort_values(ascending=False)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

filename_duplicates_spreadsheets_multispreadsheets.to_csv(f'filename_duplicates_spreadsheets_multispreadsheets_{filetime}.csv', index=False)

In [ ]:
#gemini https://gemini.google.com/app/88f42355ff50fbba
# create a df with source and target so can do this: https://frankcorso.me/chord-diagram-python-visualize-connections.html

### Chord diagram

In [ ]:
#from gemini https://gemini.google.com/app/88f42355ff50fbba and Sankey prep gemini response
from itertools import combinations

multiple_spreadsheets_chord = filename_duplicates_spreadsheets_multispreadsheets.copy()



In [ ]:
multiple_spreadsheets_chord

In [ ]:
# --- Preparation Steps ---
# 1. Split the 'filetype' string into a clean list of extensions
multiple_spreadsheets_chord['Extensions_List'] = multiple_spreadsheets_chord['filetype'].str.split(',').apply(
    lambda x: [item.strip() for item in x if item.strip()]
)
multiple_spreadsheets_chord['Extensions_List']

In [ ]:
# 2. Generate all unique combinations (pairs) for each row's list
#    combinations(list, 2) ensures pairs are not duplicated within a list (e.g., no (.ods, .ods))
multiple_spreadsheets_chord['Combinations'] = multiple_spreadsheets_chord['Extensions_List'].apply(lambda x: list(combinations(x, 2)))
multiple_spreadsheets_chord['Combinations']

In [ ]:
# 3. Explode the 'Combinations' column to create one row per pair
multiple_spreadsheets_chord_exploded = multiple_spreadsheets_chord[['id', 'Combinations']].explode('Combinations', ignore_index=True)
multiple_spreadsheets_chord_exploded

In [ ]:
# 4. Extract the Source and Target from the tuple pairs
multiple_spreadsheets_chord_exploded[['Source_Extension', 'Target_Extension']] = pd.DataFrame(
    multiple_spreadsheets_chord_exploded['Combinations'].tolist(),
    index=multiple_spreadsheets_chord_exploded.index
)
multiple_spreadsheets_chord_exploded

In [ ]:
# 5. Create a globally unique key for counting (e.g., sorted tuple)
#    This treats (.ods, .xlsx) and (.xlsx, .ods) as the same pair.
multiple_spreadsheets_chord_exploded['Unique_Pair_Key'] = multiple_spreadsheets_chord_exploded.apply(
    lambda row: tuple(sorted([row['Source_Extension'], row['Target_Extension']])),
    axis=1
)

In [ ]:
multiple_spreadsheets_chord_exploded


In [ ]:
# --- Final Expanded Table ---
multiple_spreadsheets_chord_expanded_final = multiple_spreadsheets_chord_exploded.drop(columns=['Combinations']).reset_index(drop=True)

print("--- Expanded DataFrame with Record ID ---")
print(multiple_spreadsheets_chord_expanded_final)

In [ ]:
#multiple_spreadsheets_chord_exploded_counts = multiple_spreadsheets_chord_exploded.groupby('Unique_Pair_Key')['Unique_Pair_Key'].value_counts().sort_values(ascending=False)

In [ ]:
multiple_spreadsheets_chord_exploded_counts = multiple_spreadsheets_chord_exploded.groupby('Unique_Pair_Key', sort=False).agg({'Unique_Pair_Key':'count'})

In [ ]:
multiple_spreadsheets_chord_exploded_counts.columns

In [ ]:
#reset some indexes and column names
multiple_spreadsheets_chord_exploded_counts.index.name = None
multiple_spreadsheets_chord_exploded_counts.columns=['counts']

In [ ]:
multiple_spreadsheets_chord_exploded_counts

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed = multiple_spreadsheets_chord_exploded_counts.rename_axis(['Unique_Pair_Key']).reset_index()

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed

In [ ]:
# 4. Extract the Source and Target from the tuple pairs
multiple_spreadsheets_chord_exploded_counts_reindexed[['Source_Extension', 'Target_Extension']] = pd.DataFrame(
    multiple_spreadsheets_chord_exploded_counts_reindexed['Unique_Pair_Key'].tolist(),
    index=multiple_spreadsheets_chord_exploded_counts_reindexed.index
)
multiple_spreadsheets_chord_exploded_counts_reindexed

In [ ]:
len(multiple_spreadsheets_chord_exploded_counts_reindexed['Unique_Pair_Key'].unique())

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed['Target_Extension']

In [ ]:
# 3A. Create the unique list of all nodes (LABELS) for the Chord chart
# multiple_spreadsheets_chord_exploded_counts_reindexed_all_nodes = pd.concat([multiple_spreadsheets_chord_exploded_counts_reindexed['Source_Extension'], multiple_spreadsheets_chord_exploded_counts_reindexed['Target_Extension']]).unique()
# multiple_spreadsheets_chord_exploded_counts_reindexed_labels = pd.Series(multiple_spreadsheets_chord_exploded_counts_reindexed_all_nodes).rename('Chord_Labels')

# # Create a mapping dictionary: {ID_string: index_number}
# multiple_spreadsheets_chord_exploded_counts_reindexed_label_map = {label: i for i, label in enumerate(multiple_spreadsheets_chord_exploded_counts_reindexed_all_nodes)}

In [ ]:
# 3A. Create the unique list of all nodes (LABELS) for the Sankey chart
all_nodes = pd.concat([multiple_spreadsheets_chord_exploded_counts_reindexed['Source_Extension'], multiple_spreadsheets_chord_exploded_counts_reindexed['Target_Extension']]).unique()
all_nodes

In [ ]:
df_labels = pd.Series(all_nodes).rename('Chord_Labels')

In [ ]:
df_labels

In [ ]:
# Create a mapping dictionary: {ID_string: index_number}
label_map = {label: i for i, label in enumerate(all_nodes)}

In [ ]:
label_map

In [ ]:
# 3B. Map Source and Target IDs to their corresponding indices (Source/Target Index Lists)
multiple_spreadsheets_chord_exploded_counts_reindexed['Source_Index'] = multiple_spreadsheets_chord_exploded_counts_reindexed['Source_Extension'].map(label_map)


In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed['Target_Index'] = multiple_spreadsheets_chord_exploded_counts_reindexed['Target_Extension'].map(label_map).rename('Target_Index')

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed[['Unique_Pair_Key','counts']].loc[multiple_spreadsheets_chord_exploded_counts_reindexed['counts']>1].sort_values(by='counts', ascending=False)

In [ ]:
multiple_spreadsheets_chord_exploded_counts_reindexed.plot(kind='bar', color='purple', grid=False)

In [ ]:
connections_df = multiple_spreadsheets_chord_exploded_counts_reindexed[['Source_Index','Target_Index','counts']]

In [ ]:
connections_df_holo = connections_df.rename(columns={'Source_Index':'source','Target_Index':'target','counts':'value'}).sort_values(by='value', ascending=False)

In [ ]:
connections_df_holo.sort_values(by='value', ascending=False)

In [ ]:
import holoviews as hv
from holoviews import opts, dim

hv.extension('matplotlib')
hv.output(fig='svg', size=500)

In [ ]:
hv.Chord(connections_df_holo)

In [ ]:
df_labels_dataframe = pd.DataFrame(df_labels)

In [ ]:
df_labels_dataframe

In [ ]:
#reset some indexes and column names
df_labels_dataframe.index.name = None
df_labels_dataframe.columns=['label']

In [ ]:
df_labels_dataframe

In [ ]:
nodes_ds = hv.Dataset(df_labels_dataframe, 'index')

In [ ]:
nodes_ds

In [ ]:
hv.Chord((connections_df_holo, nodes_ds)).opts(opts.Chord(
        cmap='Category20',
        edge_color=dim('source').astype(str),
        labels='label', # Make sure this matches the column name from nodes_df
        node_color=dim('index').astype(str),
        edge_linewidth=1,
        node_size=25
    )
                                         )

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement = "perpendicular",
    orientation="v",
    node = dict(
      #pad = 15,
      #thickness = 20,
      line = dict(color = "black"),
      label = df_labels_dataframe,
        # x = [0.2, 0.1, 0.5, 0.7, 0.3, 0.5],
        # y =  [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
        align="justify",
        
      color = "blue"
    ),
    link = dict(
        #arrowlen=15,
      source = connections_df_holo['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = connections_df_holo['target'],
      value = connections_df_holo['value']
  ))])

fig.update_layout(title_text="Filetypes", font_size=10, 
 autosize=False,
     width=800,
    height=900
                 )
fig.show()

In [ ]:
#used Gemini for sankey prep: https://gemini.google.com/app/596c5c70ccb22615

# --------------------------------------------------------------------------------------
# --- 3. SANKEY CHART PREPARATION: Generating Labels and Indices ---
# --------------------------------------------------------------------------------------

# 3A. Create the unique list of all nodes (LABELS) for the Sankey chart
all_nodes = pd.concat([zenodo_related_identifiers_sankey_count['relation'], zenodo_related_identifiers_sankey_count['scheme']]).unique()
df_labels = pd.Series(all_nodes).rename('Sankey_Labels')

# Create a mapping dictionary: {ID_string: index_number}
label_map = {label: i for i, label in enumerate(all_nodes)}

# 3B. Map Source and Target IDs to their corresponding indices (Source/Target Index Lists)
df_source_indices = zenodo_related_identifiers_sankey_count['relation'].map(label_map).rename('Source_Index')
df_target_indices = zenodo_related_identifiers_sankey_count['scheme'].map(label_map).rename('Target_Index')

# Output 4: List of Weights for each relationship (Plotly Values)
df_weights_only = zenodo_related_identifiers_sankey_count['count'].reset_index(drop=True).rename('Weight_List')


print("--- 3. SANKEY OUTPUT A: Unique List of All Labels (Nodes) ---")
print("This is the 'label' argument for your Plotly Sankey chart.")
print(df_labels)
print(f"\nTotal unique nodes: {len(df_labels)}\n")
print("-" * 50 + "\n")

print("--- 4. SANKEY OUTPUT B: Source Indices ---")
print("The 'source' argument for Plotly. Maps each source ID to its index in the Labels list.")
print(df_source_indices)
print(f"\nType: {type(df_source_indices)}\n")
print("-" * 50 + "\n")


print("--- 5. SANKEY OUTPUT C: Target Indices ---")
print("The 'target' argument for Plotly. Maps each target ID to its index in the Labels list.")
print(df_target_indices)
print(f"\nType: {type(df_target_indices)}\n")
print("-" * 50 + "\n")

print("--- 6. SANKEY OUTPUT D: Weights (Values) ---")
print("The 'value' argument for Plotly (frequency of the relationship).")
print(df_weights_only)
print(f"\nType: {type(df_weights_only)}\n")
print("-" * 50 + "\n")

### Records containing more than one file

In [ ]:
zenodo_files_df_deduped

In [ ]:
len(zenodo_files_df_deduped['id'].unique())

In [ ]:
#rather than looking at duplicate filenames, here we look for records that have more than one file attached
zenodo_files_df_deduped.groupby(['id'])['filetype'].apply(','.join).reset_index()

In [ ]:
zenodo_files_df_deduped.groupby(['id'])

In [ ]:
zenodo_files_df_deduped_filecounts = zenodo_files_df_deduped.groupby(['id']).agg({'filekey':'count', 'filetype': ', '.join}).sort_values(by='filekey', ascending=False )

In [ ]:
zenodo_files_df_deduped_filecounts

In [ ]:
zenodo_files_df_deduped_filecounts.columns

In [ ]:
zenodo_files_df_deduped_filecounts.index.name = None
zenodo_files_df_deduped_filecounts.columns=['counts', 'filetype']

In [ ]:
zenodo_files_df_deduped_filecounts

In [ ]:
zenodo_files_df_deduped_filecounts = zenodo_files_df_deduped_filecounts.rename_axis(['id']).reset_index()

In [ ]:
zenodo_files_df_deduped_filecounts

In [ ]:
plt.hist(zenodo_files_df_deduped_filecounts.counts, bins=50)

In [ ]:
zenodo_files_df_deduped_filecounts.drop(['filetype','id'], axis=1).plot(kind='box',legend=True,)

In [ ]:
single_files = zenodo_files_df_deduped_filecounts.loc[zenodo_files_df_deduped_filecounts['counts']==1].copy()

In [ ]:
single_files.info()

In [ ]:
single_files.groupby('filetype').size()

In [ ]:
zenodo_related_identifiers

In [ ]:
single_files

In [ ]:
single_files_related_identifiers = single_files.join(zenodo_related_identifiers.set_index('id'), on='id')

In [ ]:
#https://stackoverflow.com/questions/25050141/how-to-filter-in-nan-pandas
single_files_related_identifiers[~single_files_related_identifiers['related_identifiers'].isna()]

In [ ]:
single_files_related_identifiers[single_files_related_identifiers['related_identifiers'].isna()]

In [ ]:
#based on the counts of how many files there are on a given record, now look for records with more than one file on them
#used Google gemini to understand why I need to set as a copy
multiple_files = zenodo_files_df_deduped_filecounts.loc[zenodo_files_df_deduped_filecounts['counts']>1].copy()

In [ ]:
multiple_files.info()

In [ ]:
#random check of records with 4 files attached
zenodo_files_df_deduped_filecounts.loc[zenodo_files_df_deduped_filecounts['counts']==4]

In [ ]:
#plot of records with multiple files attached
multiple_files['counts'].plot(kind='hist', legend=True, logy=True)

In [ ]:
multiple_files

In [ ]:
#add a column that counts how many spreadsheet files there are on a record, making sure it's case insensitive
multiple_files['spreadsheet_count'] = multiple_files['filetype'].str.count(r'(\.csv|\.tsv|\.xls[xbm]?|\.xlt[xm]?|\.ods|\.ots|\.ogw)', re.I)

In [ ]:
multiple_files = multiple_files.sort_values(by='spreadsheet_count', ascending=False)

In [ ]:
multiple_files

In [ ]:
multiple_files.sort_values(by='counts',ascending=False)

In [ ]:
multiple_files.loc[multiple_files['counts']>=900].sort_values(by=['counts','spreadsheet_count'],ascending=False)

In [ ]:
multiple_files.plot.scatter(x='spreadsheet_count',
                      y='counts',
                      c='spreadsheet_count',
                      colormap='viridis')

In [ ]:
multiple_files_related_identifiers = multiple_files.join(zenodo_related_identifiers.set_index('id'), on='id')

In [ ]:
#https://stackoverflow.com/questions/25050141/how-to-filter-in-nan-pandas
multiple_files_related_identifiers[~multiple_files_related_identifiers['related_identifiers'].isna()]

In [ ]:
#look for those with more than one spreadsheet uploaded
multiple_files_spreadsheets = multiple_files.loc[multiple_files['spreadsheet_count']>1].copy()

In [ ]:
multiple_files_spreadsheets

In [ ]:
#make a plot with the percentage of spreadsheets on these multiple file records
((multiple_files_spreadsheets['spreadsheet_count'] / multiple_files_spreadsheets['counts'])*100).plot(kind='hist')

In [ ]:
multiple_files_spreadsheets['spreadsheet_count'].plot(kind='hist', legend=True, logy=True)

In [ ]:
multiple_files_spreadsheets_100plus = multiple_files_spreadsheets.loc[multiple_files_spreadsheets['spreadsheet_count']>=100]

In [ ]:
multiple_files_spreadsheets_100plus[['spreadsheet_count']].plot(kind='hist', legend=True)

In [ ]:
multiple_files_spreadsheets.loc[multiple_files_spreadsheets['spreadsheet_count']>=400]

In [ ]:
multiple_files_spreadsheets['spreadsheet_count'].loc[multiple_files_spreadsheets['spreadsheet_count']>=400].plot(kind='hist')

In [ ]:
#would like to be able to plot counts and spreadsheet_counts together
multiple_files_spreadsheets.drop(['filetype','id'], axis=1).plot(kind='box',legend=True,)

In [ ]:
multiple_files_spreadsheets.columns

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

multiple_files.to_csv(f'multiple_files_{filetime}.csv', index=False)

### File names indicating spreadsheet type

#### Figure spreadsheets

In [ ]:
#look for filenames that indicate the spreadsheet file is a figure spreadsheet
#https://note.nkmk.me/en/python-pandas-str-contains-match/
figure_filenames = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('fig[gure]*',case=False)]

In [ ]:
figure_filenames

In [ ]:
#break down figure spreadsheets by file type
figure_filenames.groupby(['filetype']).agg({'fileid':'count'}).sort_values(by='fileid', ascending=False)

In [ ]:
#export the figure spreadsheets dataframe to a CSV for analysis in OpenRefine
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

figure_filenames.to_csv(f'figure_filenames_{filetime}.csv', index=False)

#### Database files

In [ ]:
#look for database or databank filenames in spreadsheet files
database_filenames_spreadsheets = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('(data)(.?)(base|bank)',case=False)]

In [ ]:
database_filenames_spreadsheets

In [ ]:
#break down figure spreadsheets by file type
database_filenames_spreadsheets.groupby(['filetype']).agg({'fileid':'count'}).sort_values(by='fileid', ascending=False)

In [ ]:
#look for database or databank filenames that are not spreadsheets
database_filenames = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('(data)(.?)(base|bank)',case=False)]

In [ ]:
database_filenames = database_filenames.loc[~database_filenames['filetype'].str.contains('\.csv|\.tsv|\.xls[xbm]?|\.xlt[xm]?|\.ods|\.ots|\.ogw')]

In [ ]:
#break down database files by file type
database_filenames.groupby(['filetype']).agg({'fileid':'count'}).sort_values(by='fileid', ascending=False)

#### Documentation files
This includes files matching various patterns, such as including the word documentation, data dictionary, codebook, etc

In [ ]:
#look among all files for documentation filenames containing documentation, codebook, code book, data dictionary
#TO DO: also look for metadata?
documentation_filenames_spreadsheets = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('documentation|code.?book|data.?dictionar.*',case=False)]
documentation_filenames_spreadsheets

In [ ]:
documentation_filenames = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('documentation|code.?book|data.?dictionar.*',case=False)]
documentation_filenames

In [ ]:
#look for readme files, not just spreadsheet files
readme_files = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('read.?me',case=False)]
readme_files

In [ ]:
#look for files containing text about, of any type
#TO DO: check these results and drop rows that are not actually documentation files
about_files = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('about',case=False)]
about_files

In [ ]:
#look for files containing the word information (in several languages)
#TO DO: check these results and drop rows that are not actually documentation files
information_files = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('informa.ion.?',case=False)]
information_files

## File name best practices

### Use short file names
Checks for filenames with extensions that are too long, too short, and at best practice length

In [ ]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext_len']>=200]

In [ ]:
len(zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext_len']>=200])

In [ ]:
# 32 characters, as recommended in https://caltechlibrary.github.io/RDMworkbook/file-organization-and-naming.html#file-naming
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext_len']>32]

In [ ]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext_len'] < 4 ]

In [ ]:
#https://www.geeksforgeeks.org/python/filter-pandas-dataframe-with-multiple-conditions/
zenodo_spreadsheet_files_df.loc[(zenodo_spreadsheet_files_df['filename_noext_len']>=4) & (zenodo_spreadsheet_files_df['filename_noext_len']<=32)]

### Use camel case (e.g, RainAvg) in file names.
Using caseutil to check for cases

In [ ]:
#add a column to the spreadsheet files dataframe that indicates what case caseutils thinks the filename with no extension is
zenodo_spreadsheet_files_df['filename_cases'] = zenodo_spreadsheet_files_df['filename_noext'].apply(lambda x: filename_cases(x))

In [ ]:
zenodo_spreadsheet_files_df

In [ ]:
#where caseutil couldn't determine a case, fill those rows with value 'mixed'
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_cases'] == '', 'filename_cases'] = 'mixed'

In [ ]:
zenodo_spreadsheet_files_groups = zenodo_spreadsheet_files_df.groupby(['filename_cases'])

In [ ]:
zenodo_spreadsheet_files_groups

In [ ]:
#get counts of the different cases present
zenodo_filename_cases_counts = zenodo_spreadsheet_files_groups.agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

In [ ]:
zenodo_filename_cases_counts

In [ ]:
zenodo_spreadsheet_files_groups.get_group(('lower;upper',))

In [ ]:
#export csv files for each of the caseutil determinations, so they can be analyzed in OpenRefine
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
for key in zenodo_spreadsheet_files_groups.groups.keys():
    print(key)
    zenodo_spreadsheet_files_groups_cases = zenodo_spreadsheet_files_groups.get_group((key,))
    newkey = key.replace(';','_')
    print(newkey)
    zenodo_spreadsheet_files_groups_cases.to_csv(f'zenodo_cases_{newkey}_{filetime}.csv', index=False)

### Never include "final" in a file name

In [ ]:
#look for filenames that contain some variation of the word final
final_filenames_spreadsheets = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('final.?',case=False)]

In [ ]:
final_filenames_spreadsheets

In [ ]:
#look for files, not just spreadsheets, that contain the word final
final_filenames = zenodo_files_df_deduped.loc[zenodo_files_df_deduped['filename_noext'].str.contains('final.?',case=False)]

In [ ]:
#group filenames containing the word final by file type
final_filenames.groupby('filetype').agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

### Examine spaces and special characters

In [ ]:
#Look for the following special characters: space\.\$@%#&\*\!\(\)\{\}\[\]~\^`;<>,\?\'\"\|–—\=
zenodo_spreadsheets_spaces_specialchars = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('[ \.\$@%#&\*\!\(\)\{\}\[\]~\^`;<>,\?\'\"\|–—\=]',case=False)].copy()

In [ ]:
#add an empty column for the kind of special character(s) found
zenodo_spreadsheets_spaces_specialchars['char'] = ''

In [ ]:
zenodo_spreadsheets_spaces_specialchars

In [13]:
#create a list of tuples containing the regex for a special character, plus the label for that character

#https://blog.finxter.com/5-best-ways-to-create-a-list-of-tuples-from-csv-in-python/
special_chars_list_df = pd.read_csv('special-chars-2025-12-16.csv')
tuples_list = [tuple(x) for x in special_chars_list_df.to_numpy()]
print(tuples_list)

#characters = [('[ ]','space'),('[\.]','period'),('[\$]','dollar'),('[@]','at'),('[%]','percent'),('[#]','pound'),('[&]','ampersand'),('[\*]','asterisk'),('[\!]','exclamation'),('[\(\)]','parentheses'),('[\{\}]','curly-bracket'),('[\[\]]','square-bracket'),('[~]','tilde'),('[\^]','caret'),('[`]','backtick'),('[;]','semicolon'),('[<>]','angle-bracket'),('[,]','comma'),('[\?]','question'),('[\\\']','single-quote'),('[\\\"]','double-quote'),('[\|]','pipe'),('[–]','en-dash'),('[—]','em-dash'),('[\=]','equals'),]


[('[&]', 'ampersand'), ('[<>]', 'angle-bracket'), ('[\\*]', 'asterisk'), ('[@]', 'at'), ('[`]', 'backtick'), ('[\\^]', 'caret'), ('[,]', 'comma'), ('[\\{\\}]', 'curly-bracket'), ('[°]', 'degree'), ('[\\$]', 'dollar'), ('[\\\\\\"]', 'double-quote'), ('[—]', 'em-dash'), ('[✅]', 'emoji-check'), ('[❌]', 'emoji-x'), ('[–]', 'en-dash'), ('[\\=]', 'equals'), ('[\\!]', 'exclamation'), ('[≥]', 'greater-equal'), ('[α]', 'greek-alpha'), ('[β]', 'greek-beta'), ('[δ]', 'greek-delta'), ('[γ]', 'greek-gamma'), ('[λ]', 'greek-lambda'), ('[µ]', 'greek-mu'), ('[ν]', 'greek-nu'), ('[ο]', 'greek-omicron'), ('[σ]', 'greek-sigma'), ('[τ]', 'greek-tau'), ('[∆]', 'increment'), ('[【】]', 'lenticular-bracket'), ('[ª]', 'ordinal-a'), ('[º]', 'ordinal-o'), ('[\\(\\)]', 'parentheses'), ('[%]', 'percent'), ('[\\.]', 'period'), ('[\\|]', 'pipe'), ('[+]', 'plus'), ('[±]', 'plus-minus'), ('[#]', 'pound'), ('[\\?]', 'question'), ('[�]', 'question-diamond'), ('[;]', 'semicolon'), ("[\\\\\\']", 'single-quote'), ('[\\\\\\/

In [ ]:
#loop through the list of tuples
for char_regex, label in characters:
    print(char_regex,label)
    #find rows where filename contains the special character, then add the label to the char column
    zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains(f'{char_regex}',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + f'{label};'

In [ ]:
zenodo_spreadsheets_spaces_specialchars['char']

In [ ]:
zenodo_spreadsheets_spaces_specialchars['char'] = zenodo_spreadsheets_spaces_specialchars['char'].str.replace(';$','',regex=True)

In [ ]:
#create dataframe to count the occurrences of the various special character combos
zenodo_spreadsheets_spaces_specialchars_combinations = zenodo_spreadsheets_spaces_specialchars.groupby('char').agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

In [ ]:
zenodo_spreadsheets_spaces_specialchars_combinations.index.name = None
zenodo_spreadsheets_spaces_specialchars_combinations.columns=['counts']

In [ ]:
zenodo_spreadsheets_spaces_specialchars_combinations

In [ ]:
zenodo_spreadsheets_spaces_specialchars_combinations = zenodo_spreadsheets_spaces_specialchars_combinations.rename_axis(['chars']).reset_index()

In [ ]:
#for readability, create a new column called chars that replaces the semicolons with commas and spaces
zenodo_spreadsheets_spaces_specialchars_combinations['chars'] = zenodo_spreadsheets_spaces_specialchars_combinations['chars'].str.replace(';',', ')

In [ ]:
zenodo_spreadsheets_spaces_specialchars_combinations

In [ ]:
#explode the char column so we can count how many times each character occurs
zenodo_spreadsheets_spaces_specialchars_explode = zenodo_spreadsheets_spaces_specialchars.assign(char=zenodo_spreadsheets_spaces_specialchars.char.str.split(';')).explode('char')

In [ ]:
zenodo_spreadsheets_spaces_specialchars_explode.groupby(['char']).agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

### Examine file names with no spaces or special characters

In [ ]:
#find the rows where the filenames do not contain any special characters
zenodo_spreadsheets_no_spaces_specialchars = zenodo_spreadsheet_files_df.loc[~zenodo_spreadsheet_files_df['filename_noext'].str.contains('[ \.\$@%#&\*\!\(\)\{\}\[\]~\^`;<>,\?\'\"\|–—\=]',case=False)].copy()

In [ ]:
zenodo_spreadsheets_no_spaces_specialchars

In [ ]:
#add a placeholder char column to later record what kind of non-special character is present
zenodo_spreadsheets_no_spaces_specialchars['char'] = ''

In [ ]:
zenodo_spreadsheets_no_spaces_specialchars

In [ ]:
#look for filenames with no hyphens or underscores and fill those rows with the word 'none'
zenodo_spreadsheets_no_spaces_specialchars.loc[~zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('[_-]',case=False), 'char'] = zenodo_spreadsheets_no_spaces_specialchars['char'] + 'none'

In [ ]:
zenodo_spreadsheets_no_spaces_specialchars.loc[~zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('[_-]',case=False)]

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
#look for various combinations of hyphens and underscores and add the type to the char column
hyphen_underscore_regexes = [('^(?=.*_)(?:(?!-).)*$','underscore'),('^(?=.*-)(?:(?!_).)*$','hyphen'),('^(?=.*-)(?=.*_).*$','both')]
for hyphen_underscore_regex, label in hyphen_underscore_regexes:
    print(hyphen_underscore_regex,label)
    zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains(f'{hyphen_underscore_regex}',case=False), 'char'] = zenodo_spreadsheets_no_spaces_specialchars['char'] + f'{label}'

In [ ]:
#get a count of occurrences of the different best practice characters (none, hyphens, underscores, or both)
zenodo_spreadsheets_no_spaces_specialchars.groupby('char').agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

In [ ]:
#get a count of occurrences of the different best practice characters (none, hyphens, underscores, or both)
zenodo_spreadsheets_no_spaces_specialchars.groupby('char').agg({'fileid':'count'}).sort_values(by='fileid',ascending=False).plot(kind='pie', subplots=True)

### Look at special and best practice characters together

In [ ]:
#https://www.geeksforgeeks.org/how-to-concatenate-two-or-more-pandas-dataframes/
zenodo_spreadsheets_chars = pd.concat([zenodo_spreadsheets_no_spaces_specialchars, zenodo_spreadsheets_spaces_specialchars],axis=0,ignore_index=True)

In [ ]:
zenodo_spreadsheets_chars['capitalization'] = ''

In [ ]:
zenodo_spreadsheets_chars['capitalization'] = zenodo_spreadsheets_chars['filename_noext'].apply(lambda x: filename_capitalization(x))

In [ ]:
zenodo_spreadsheets_chars

In [ ]:
zenodo_spreadsheets_chars_counts = zenodo_spreadsheets_chars.groupby('char').agg({'fileid':'count'}).sort_values(by='fileid',ascending=False)

In [ ]:
#get a combined count of identified characters in filenames
#https://btechgeeks.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
print(zenodo_spreadsheets_chars_counts.to_string())

In [ ]:
print(zenodo_spreadsheets_chars.groupby(['filename_cases','char','capitalization']).agg({'fileid':'count'}).sort_values(by=['filename_cases','fileid'],ascending=False).to_string())

In [ ]:
print(zenodo_spreadsheets_chars.groupby(['capitalization','char','filename_cases']).size().sort_values(ascending=False).to_string())

In [ ]:
zenodo_spreadsheets_chars_common = zenodo_spreadsheets_chars.groupby(['capitalization','char','filename_cases']).size().sort_values(ascending=False)

In [ ]:
zenodo_spreadsheets_chars_common

In [ ]:
zenodo_spreadsheets_chars_common_graph = zenodo_spreadsheets_chars_common.loc[zenodo_spreadsheets_chars.groupby(['capitalization','char','filename_cases']).size() > 7000].plot(kind='bar', color='purple', ylabel='Number of instances', xlabel='Cases and special chars', grid=True)

#annotate bars
zenodo_spreadsheets_chars_common_graph.bar_label(zenodo_spreadsheets_chars_common_graph.containers[0])

### Language of file names

In [ ]:
zenodo_spreadsheet_files_df

In [ ]:
#replace hyphens and underscores with spaces for langdetect process
zenodo_spreadsheets_chars['filename_noext_spaces'] = zenodo_spreadsheets_chars['filename_noext'].str.replace('_|-',' ',regex=True)

In [ ]:
zenodo_spreadsheets_chars

In [ ]:
#even optimized this takes 20 minutes
zenodo_spreadsheets_chars['filename_lang'] = zenodo_spreadsheets_chars['filename_noext'].swifter.apply(detect_lang)
#df['language'] = df['text_column'].swifter.apply(detect_lang)

In [ ]:
#even optimized this takes 20 minutes
zenodo_spreadsheets_chars['filename_lang_spaces'] = zenodo_spreadsheets_chars['filename_noext_spaces'].swifter.apply(detect_lang)
#df['language'] = df['text_column'].swifter.apply(detect_lang)

In [ ]:
zenodo_spreadsheets_chars_langs = zenodo_spreadsheets_chars.groupby('filename_lang').size().sort_values(ascending=False)

In [ ]:
zenodo_spreadsheets_chars_langs

In [ ]:
zenodo_spreadsheets_chars

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheets_chars['filename_lang'].to_csv(f'zenodo_spreadsheets_chars_filename_lang_{filetime}.csv')

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheets_chars.to_csv(f'zenodo_spreadsheets_chars_{filetime}.csv')

## Create lists of record ids

This section generates a list of all spreadsheet files, which can be used with zenodo_get to download all files. It also creates a sample of Excel and ODS files and a random sample of all files.

### Random sample of 300 records

In [ ]:
#randomly sample from 300 records
#make a dataframe of just the spreadsheet files
#from each record, select one spreadsheet file; "if you've already seen this record id, skip this one'
#groupby record id, then take one per

In [ ]:
zenodo_df_300recs = zenodo_df_deduped.sample(n=300, random_state=18)

In [ ]:
zenodo_df_300recs['id']

In [ ]:
#get the unique record ids for these files
zenodo_df_300recs_ids = zenodo_df_300recs['id'].unique()

In [ ]:
zenodo_spreadsheet_files_df_300recs = zenodo_spreadsheet_files_df.query('id in @zenodo_df_300recs_ids')

In [ ]:
#take just one spreadsheet file from each record
zenodo_spreadsheet_files_df_300recs_sample = zenodo_spreadsheet_files_df_300recs.groupby('id').sample(n=1, random_state=1)

In [ ]:
zenodo_spreadsheet_files_df_300recs_sample.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']})

In [ ]:
zenodo_spreadsheet_files_df_300recs_sample['filesize_gb'].sum()

In [ ]:
#get the count of each filetype as percentages
zenodo_spreadsheet_files_df_300recs_sample.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']}).transform(lambda x: (x / 300) * 100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_df_300recs_sample.to_csv(f'zenodo_spreadsheet_files_df_300recs_sample_{filetime}.csv', index_label='df_index')

### Random sample of 100 records

In [ ]:
#randomly sample from 100 records
#make a dataframe of just the spreadsheet files
#from each record, select one spreadsheet file; "if you've already seen this record id, skip this one'
#groupby record id, then take one per

In [ ]:
zenodo_df_100recs = zenodo_df_deduped.sample(n=100, random_state=10)

In [ ]:
zenodo_df_100recs['id']

In [ ]:
#get the unique record ids for these files
zenodo_df_100recs_ids = zenodo_df_100recs['id'].unique()

In [ ]:
zenodo_spreadsheet_files_df_100recs = zenodo_spreadsheet_files_df.query('id in @zenodo_df_100recs_ids')

In [ ]:
#take just one spreadsheet file from each record
zenodo_spreadsheet_files_df_100recs_sample = zenodo_spreadsheet_files_df_100recs.groupby('id').sample(n=1, random_state=1)

In [ ]:
zenodo_spreadsheet_files_df_100recs_sample.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']})

In [ ]:
zenodo_spreadsheet_files_df_100recs_sample['filesize_gb'].sum()

In [ ]:
#get the count of each filetype as percentages
zenodo_spreadsheet_files_df_100recs_sample.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']}).transform(lambda x: (x / 100) * 100)

In [ ]:
zenodo_spreadsheet_files_df_100recs_sample.loc[zenodo_spreadsheet_files_df_100recs_sample['filetype']=='.csv']

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_df_100recs_sample.to_csv(f'zenodo_spreadsheet_files_df_100recs_sample_{filetime}.csv', index_label='df_index')

### Make lists for zenodo_get

In [ ]:
#make one text file of all ids
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
zenodo_spreadsheet_files_ids = zenodo_spreadsheet_files_df['id'].unique()
np.savetxt(f'zenodo_spreadsheet_file_ids_{filetime}.csv', zenodo_spreadsheet_files_ids, fmt='%d')

In [ ]:
zenodo_spreadsheet_files_ids

In [ ]:
#make text files of ids by file type
zenodo_spreadsheet_files_df.groupby(['filetype'])

In [ ]:
#want to make a list of ids for each subgroup
zenodo_spreadsheet_files_df.groupby(['filetype']).groups.keys()
#zenodo_spreadsheet_files_df.iloc[gb.groupby.indices.get(filetype)]

In [ ]:
zenodo_spreadsheet_files_csv_ids = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filetype'] == '.csv']['id']

In [ ]:
zenodo_spreadsheet_files_csv_ids

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_csv_ids.to_csv(f'csv_ids_{filetime}.csv', index=False, header=False)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
for key in zenodo_spreadsheet_files_df.groupby(['filetype']).groups.keys():
#zenodo_spreadsheet_files_df.groupby(['filetype']).groups.keys():
    print(key)
    zenodo_spreadsheet_files_ids = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filetype'] == key]['id'].unique()
    #print(zenodo_spreadsheet_files_ids)
    #print(zenodo_spreadsheet_files_ids)
    newkey = key.strip('.')
    print(newkey)
    np.savetxt(f'{newkey}_ids_{filetime}.csv', zenodo_spreadsheet_files_ids, fmt='%d')
    # print(f'{key}_ids_{filetime}.csv')

# Old code

### Create samples of 100 records for selected filetypes

In [ ]:
zenodo_spreadsheet_groups = zenodo_spreadsheet_files_df.groupby(['filetype'])

In [ ]:
zenodo_spreadsheet_groups

In [ ]:
csv_group = zenodo_spreadsheet_groups.get_group(('.csv',))
csv_group

In [ ]:
csv_sample = csv_group.sample(n=100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

csv_sample.to_csv(f'csv_sample_ids_{filetime}.csv')

In [ ]:
tsv_group = zenodo_spreadsheet_groups.get_group(('.tsv',))
tsv_group

In [ ]:
tsv_sample = tsv_group.sample(n=100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

tsv_sample.to_csv(f'tsv_sample_ids_{filetime}.csv')

In [ ]:
xls, xlsm, ods

In [ ]:
xls_group = zenodo_spreadsheet_groups.get_group(('.xls',))
xls_group

In [ ]:
xls_sample = xls_group.sample(n=100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

xls_sample.to_csv(f'xls_sample_ids_{filetime}.csv')

In [ ]:
xlsm_group = zenodo_spreadsheet_groups.get_group(('.xlsm',))
xlsm_group

In [ ]:
xlsm_sample = xlsm_group.sample(n=100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

xlsm_sample.to_csv(f'xlsm_sample_ids_{filetime}.csv')

In [ ]:
ods_group = zenodo_spreadsheet_groups.get_group(('.ods',))
ods_group

In [ ]:
ods_sample = ods_group.sample(n=100)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

ods_sample.to_csv(f'ods_sample_ids_{filetime}.csv')

In [ ]:
#period
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\.',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'period;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\.',case=False)]

In [ ]:
#space
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains(' ',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'space;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains(' ',case=False)]

In [ ]:
#dollar
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\$',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'dollar;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\$',case=False)]

In [ ]:
# at symbol
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('@',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'at;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('@',case=False)]

In [ ]:
# percent sign
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('%',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'percent;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('%',case=False)]

In [ ]:
#pound sign
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('#',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'pound;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('#',case=False)]

In [ ]:
# ampersand
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('&',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'ampersand;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('&',case=False)]

In [ ]:
#asterisk
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\*',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'asterisk;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\*',case=False)]

In [ ]:
#parentheses
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\(\)]',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'parentheses;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\(\)]',case=False)]

In [ ]:
#curly bracket
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\{\}]',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'curly-bracket;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\{\}]',case=False)]

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\[\]]',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'square-bracket;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('[\[\]]',case=False)]

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\!',case=False), 'char'] = zenodo_spreadsheets_spaces_specialchars['char'] + 'exclamation;'

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\!',case=False)]

In [ ]:
zenodo_spreadsheets_spaces_specialchars.loc[zenodo_spreadsheets_spaces_specialchars['filename_noext'].str.contains('\!',case=False)]

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for just underscores, no hyphens
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*_)(?:(?!-).)*$',case=False), 'char'] = zenodo_spreadsheets_no_spaces_specialchars['char'] + 'underscore'

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for just underscores, no hyphens
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*_)(?:(?!-).)*$',case=False)]

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for just hyphens, no underscores
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*-)(?:(?!_).)*$',case=False), 'char'] = zenodo_spreadsheets_no_spaces_specialchars['char'] + 'hyphen'

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for just hyphens, no underscores
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*-)(?:(?!_).)*$',case=False)]

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for both hyphens and underscores
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*-)(?=.*_).*$',case=False), 'char'] = zenodo_spreadsheets_no_spaces_specialchars['char'] + 'both'

In [ ]:
#https://superuser.com/questions/1335621/creating-a-regex-expression-with-a-not-condition-for-a-specific-substring
# look for both hyphens and underscores
zenodo_spreadsheets_no_spaces_specialchars.loc[zenodo_spreadsheets_no_spaces_specialchars['filename_noext'].str.contains('^(?=.*-)(?=.*_).*$',case=False)]

In [ ]:
#https://github.com/pandas-dev/pandas/issues/8121#issuecomment-53578872
#gr = zenodo_spreadsheet_files_df.groupby(pd.factorize(zenodo_spreadsheet_files_df.filename_cases)[0])

In [ ]:
#gr.get_group(16)

### Make list for testing other scripts
List will be used to test zenodo_get bash script, Jupyter notebook for creating the downloaded file inventory, and code in this notebook for comparing metadata and files dataframes to inventory

In [ ]:
#take a random sample of files
zenodo_spreadsheet_files_df_sample = zenodo_spreadsheet_files_df.groupby(['filetype']).sample(frac=0.0001)

In [ ]:
zenodo_spreadsheet_files_df_sample

In [ ]:
zenodo_spreadsheet_files_df_sample.groupby(['filetype']).agg({'id': 'count'})

In [ ]:
#get the unique record ids for these files
zenodo_spreadsheet_files_df_sample_ids = zenodo_spreadsheet_files_df_sample['id'].unique()

In [ ]:
#count how many unique record ids there actually are
len(zenodo_spreadsheet_files_df_sample_ids)

In [ ]:
#using that unique list of ids, make a dataframe listing all the spreadsheet files associated with these ids
#this is more than the sample of 100, since many records have more than one spreadsheet file uploaded
zenodo_spreadsheet_files_df_sample_all = zenodo_spreadsheet_files_df.query('id in @zenodo_spreadsheet_files_df_sample_ids')

In [ ]:
zenodo_spreadsheet_files_df_sample_all

In [ ]:
len(zenodo_spreadsheet_files_df_sample_all.index)

In [ ]:
len(zenodo_spreadsheet_files_df_sample_all['id'].unique())

In [ ]:
#make a dataframe that counts how many spreadsheets there are uploaded to each record by grouping by record id.
#filter down to those with fewer than 10 files
zenodo_spreadsheet_files_df_sample_all_counts = zenodo_spreadsheet_files_df_sample_all.groupby(['id']).count() <= 10

In [ ]:
#get just the ones that evaluate as "true"
#https://www.statology.org/pandas-filter-by-boolean-column/
zenodo_spreadsheet_files_df_sample_all_counts.loc[zenodo_spreadsheet_files_df_sample_all_counts.fileid]

In [ ]:
#put the ones with 10 or fewer spreadsheets in a dataframe
zenodo_spreadsheet_files_df_sample_all_counts_small = zenodo_spreadsheet_files_df_sample_all_counts.loc[zenodo_spreadsheet_files_df_sample_all_counts.fileid].index

In [ ]:
len(zenodo_spreadsheet_files_df_sample_all_counts_small)

In [ ]:
#use that list of records with fewer than 10 spreadsheets uploaded to filter files
zenodo_spreadsheet_files_df_sample_records_small = zenodo_spreadsheet_files_df_sample_all.query('id in @zenodo_spreadsheet_files_df_sample_all_counts_small')
#.groupby('filetype').groups.keys()

In [ ]:
len(zenodo_spreadsheet_files_df_sample_records_small.index)

In [ ]:
zenodo_spreadsheet_files_df_sample_records_small

In [ ]:
zenodo_spreadsheet_files_df_sample_records_small_size = zenodo_spreadsheet_files_df_sample_records_small['filesize'].sum()
print(format_bytes(zenodo_spreadsheet_files_df_sample_records_small_size, 'GB', SI=True))

In [ ]:
zenodo_spreadsheet_files_df_sample_records_small.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']})

In [ ]:
zenodo_spreadsheet_files_df_sample_records_small_ids = zenodo_spreadsheet_files_df_sample_records_small['id'].unique()

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
np.savetxt(f'zenodo_spreadsheet_files_df_sample_records_small_ids_{filetime}.csv', zenodo_spreadsheet_files_df_sample_records_small_ids, fmt='%d')

### Take a random sample of files for file-level analysis

In [ ]:
#take a random sample of files
zenodo_spreadsheet_files_df_random_sample01 = zenodo_spreadsheet_files_df.groupby(['filetype']).sample(frac=0.01, random_state = 1)

In [ ]:
#zenodo_spreadsheet_files_df_random_sample01_size = 
zenodo_spreadsheet_files_df_random_sample01['filesize_gb'].sum()
#print(format_bytes(zenodo_spreadsheet_files_df_sample_records_small_size, 'GB', SI=True))

In [ ]:
#take a random sample of files
zenodo_spreadsheet_files_df_random_sample001 = zenodo_spreadsheet_files_df.groupby(['filetype']).sample(frac=0.001, random_state = 1)

In [ ]:
#zenodo_spreadsheet_files_df_random_sample01_size = 
zenodo_spreadsheet_files_df_random_sample001['filesize_gb'].sum()
#print(format_bytes(zenodo_spreadsheet_files_df_sample_records_small_size, 'GB', SI=True))

In [ ]:
zenodo_spreadsheet_files_df_random_sample001

In [ ]:
zenodo_spreadsheet_files_df_random_sample001.groupby(['filetype']).agg({'filetype': ['count'], 'filesize_gb': ['sum']})

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_df_random_sample001.to_csv(f'zenodo_spreadsheet_files_df_random_sample001_{filetime}.csv', index_label='df_index')

In [ ]:
zenodo_spreadsheet_files_df_random_sample01.describe()

In [ ]:
#get the unique record ids for these files
#zenodo_spreadsheet_files_df_random_sample001_ids = 
zenodo_spreadsheet_files_df_random_sample001['id'].unique()

In [ ]:
for link in zenodo_spreadsheet_files_df_random_sample001['filelinks.self']:
    print(link)

## sampling

In [ ]:
#https://www.w3schools.in/python/examples/language-detection
#https://stackoverflow.com/questions/39142778/how-to-determine-the-language-of-a-piece-of-text
x='Subir_Seleccion_Maquinal'
detect_lang(x)
#print(languages)
# try:
#     languages = detect_langs(x)
#     lang_string = ''
#     for language in languages:
#         lang_code = language.lang
#         #print(lang_code)
#         lang_prob = str(language.prob)
#         #print(lang_prob)
#         lang_string += lang_code +';'+lang_prob +'|'
#     print(lang_string)
# except Exception as e:
#     print(e)
# # Sample Output:
# # en 0.50 (English)
# # hi 0.50 (Hindi)




In [ ]:
excel_ods_filetypes = ['.xls','.xlsb','.xlsx','.xltx','.ods','.ots','.xlsm','.xltm','.xltx']

In [ ]:
#https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
excel_ods_files = zenodo_spreadsheet_files_df.query('filetype in @excel_ods_filetypes')

In [ ]:
excel_ods_files

In [ ]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
excel_ods_filesize_total = excel_ods_files['filesize'].sum()
print(excel_ods_filesize_total)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

excel_ods_files['id'].to_csv(f'excel_ods_{filetime}.csv', index=False)

In [ ]:
# #filetime = datetime.now()
# # filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

# zenodo_spreadsheet_files_df['id'].to_csv(f'zenodo_spreadsheet_files_{filetime}.csv', index=False)

In [ ]:
#take a random sample of a small fraction of files files
#zenodo_spreadsheet_files_df_sample = zenodo_spreadsheet_files_df.groupby(['filetype']).sample(frac=0.0001)

In [ ]:
unique_spreadsheet_sample_files_ids = zenodo_spreadsheet_files_df_sample_records_small.loc[zenodo_spreadsheet_files_df_sample_records_small['filetype'] == '.tsv']['id'].unique()

In [ ]:
unique_spreadsheet_sample_files_ids

In [ ]:

zenodo_spreadsheet_sample_files_ids = zenodo_spreadsheet_files_df_sample_records_small.loc[zenodo_spreadsheet_files_df_sample_records_small['filetype'] == key]['id'].unique()


In [ ]:
np.savetxt('unique_spreadsheet_sample_files_ids.txt', unique_spreadsheet_sample_files_ids, fmt='%d')

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')
for key in zenodo_spreadsheet_files_df_sample_records_small.groupby(['filetype']).groups.keys():
#zenodo_spreadsheet_files_df.groupby(['filetype']).groups.keys():
    print(key)
    zenodo_spreadsheet_sample_files_ids = zenodo_spreadsheet_files_df_sample_records_small.loc[zenodo_spreadsheet_files_df_sample_records_small['filetype'] == key]['id'].unique()
    #print(zenodo_spreadsheet_files_ids)
    newkey = key.strip('.')
    print(newkey)
    #https://www.geeksforgeeks.org/how-to-save-a-numpy-array-to-a-text-file/
    np.savetxt(f'{newkey}_small_sample_ids_{filetime}.csv', zenodo_spreadsheet_sample_files_ids, fmt='%d')
    # print(f'{key}_ids_{filetime}.csv')

In [ ]:
excel_ods_files_sample = excel_ods_files.sample(n=100, random_state=1)

In [ ]:
excel_ods_files_sample

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

excel_ods_files_sample['id'].to_csv(f'excel_ods_files_sample_{filetime}.csv', index=False)

In [ ]:
zenodo_spreadsheet_files_df_sample_percentage = zenodo_spreadsheet_files_df.sample(frac=0.01)

In [ ]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
len(zenodo_spreadsheet_files_df_sample_percentage.index)

In [ ]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
zenodo_spreadsheet_files_df_sample_percentage_size = zenodo_spreadsheet_files_df_sample_percentage['filesize'].sum()
print(format_bytes(zenodo_spreadsheet_files_df_sample_percentage_size, 'GB', SI=True))

In [ ]:
zenodo_spreadsheet_files_df_sample_percentage_ids = zenodo_spreadsheet_files_df_sample_percentage['id'].unique()

In [ ]:
zenodo_spreadsheet_files_df_sample_percentage_all = zenodo_spreadsheet_files_df.query('id in @zenodo_spreadsheet_files_df_sample_percentage_ids')

In [ ]:
len(zenodo_spreadsheet_files_df_sample_percentage_all.index)

In [ ]:
zenodo_spreadsheet_files_df_sample_percentage_all_size = zenodo_spreadsheet_files_df_sample_percentage_all['filesize'].sum()
print(format_bytes(zenodo_spreadsheet_files_df_sample_percentage_all_size, 'GB', SI=True))

In [ ]:
zenodo_spreadsheet_files_df_sample_percentage_all.groupby(['filetype']).agg({'filetype': ['count']})

In [ ]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['id'] == 12805219]